In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import yaml

In [2]:
df_subdist_shape = gpd.read_file('/home/inductive_anks/Backup Files/CS and AI/Internship/Data/All-India-Subdistrict-Shape-File/SUBDISTRICT_BOUNDARY.shp')

In [3]:
df_village_list = pd.read_csv('/home/inductive_anks/Machine Learning/Yield-Prediction/Internship/data/Tehsil Data/Tehsil_Data_With_Single_Village.csv')

In [4]:
df_SOI_shape = gpd.read_file('/home/inductive_anks/Machine Learning/Yield-Prediction/Internship/data/All-Inida-SOI/All-India-Villages-SOI.shp')

In [19]:
df_SOI_shape.shape

(599742, 5)

In [20]:
df_SOI_shape.head()

,NAME,dtnmcoc,state,tehnmcoc,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [22]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,IV,Puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,V,Sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,V,Zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,I,Etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,I,Tekkali,2,tirlangi


In [9]:
df_subdist_shape.head()

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,BIL>SPUR,HIMACHAL PRADESH,BHARARI,50253.228763,9.000306e+07,"POLYGON ((3690806.89 4831388.917, 3690871.421 ..."
1,BIL>SPUR,HIMACHAL PRADESH,BIL>SPUR,93446.685818,1.703525e+08,"POLYGON ((3699861.726 4812292.061, 3699965.144..."
2,BIL>SPUR,HIMACHAL PRADESH,GHURMW|N,105834.665205,1.906542e+08,"POLYGON ((3692609.535 4827764.245, 3692614.853..."
3,BIL>SPUR,HIMACHAL PRADESH,JHAND@TA,112831.475031,3.205376e+08,"POLYGON ((3667564.424 4819909.775, 3667603.174..."
4,BIL>SPUR,HIMACHAL PRADESH,NAMHOL,61752.616994,1.034979e+08,"POLYGON ((3701617.644 4803943.439, 3701666.832..."


## Converting to Lower Case

In [25]:
df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_6758/228283043.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_subdist_shape = df_subdist_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [26]:
df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_6758/990218482.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_village_list = df_village_list.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [27]:
df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_6758/3172920881.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_SOI_shape = df_SOI_shape.applymap(lambda x: x.lower() if isinstance(x, str) else x)


## Using Jaro Similarity to Filter SOI Tehsils using Village List

In [28]:
df_village_list.head()

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
0,2008-2011,andhra pradesh,ranga reddy,iv,puduru,40,pedhaumethala
1,2008-2011,andhra pradesh,medak,v,sadasivpet,49,nandi kandi
2,2008-2011,andhra pradesh,medak,v,zaiheerabad,50,huggelly
3,2008-2011,andhra pradesh,srikakulam,i,etheria,1,ponnada
4,2008-2011,andhra pradesh,srikakulam,i,tekkali,2,tirlangi


In [39]:
df_SOI_shape.sample(6)

,NAME,dtnmcoc,state,tehnmcoc,geometry
430346,karri,chhatarpur,madhya pradesh,bijawar,"POLYGON ((79.29296 24.17866, 79.29297 24.17864..."
501538,rathauranpura,jalaun,uttarpradesh,madhogarh,"POLYGON ((79.09735 26.37127, 79.09733 26.3708,..."
106018,bhada,purba champaran,bihar,harsidhi,"POLYGON ((84.72984 26.56127, 84.7336 26.56049,..."
233719,sinjo,latehar,jharkhand,manika,"POLYGON ((84.37235 23.83692, 84.37296 23.83445..."
573533,gulra,kheri,uttarpradesh,palia kalan,"POLYGON ((80.69836 28.39441, 80.69823 28.3938,..."
31437,dharur,mahbubnagar,andhra pradesh,dharur,"POLYGON ((77.69743 16.25062, 77.69795 16.2508,..."


In [42]:
import pandas as pd
from nltk.metrics.distance import jaro_similarity

In [62]:
def calculate_jaro_similarity(village_name1, village_name2):
    if pd.isnull(village_name1) or pd.isnull(village_name2):
        return 0
    return jaro_similarity(village_name1, village_name2)

In [48]:
df_SOI_shape['similarity'] = df_SOI_shape.apply(
    lambda row: max([calculate_jaro_similarity(row['NAME'], village) for village in df_village_list['Village Name']]),
    axis=1
)

In [49]:
df_SOI_shape.head(10)

,NAME,dtnmcoc,state,tehnmcoc,geometry,similarity
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101...",0.762626
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775...",0.785714
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418...",0.813492
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894...",0.757835
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541...",0.777778
5,dhar topiyun,chamba,himachal pradesh,pangi,"POLYGON ((76.71984 33.18102, 76.72248 33.17966...",0.785714
6,tai,chamba,himachal pradesh,pangi,"POLYGON ((76.43819 33.17915, 76.44485 33.17832...",0.791667
7,chaloli,chamba,himachal pradesh,pangi,"POLYGON ((76.37752 33.16003, 76.38194 33.16366...",0.849206
8,kuthah,chamba,himachal pradesh,pangi,"POLYGON ((76.37041 33.15255, 76.37295 33.15312...",0.819444
9,luj,chamba,himachal pradesh,pangi,"POLYGON ((76.37041 33.15255, 76.37119 33.1482,...",0.688889


In [50]:
df_SOI_shape_filtered = df_SOI_shape[df_SOI_shape['similarity'] > 0.95]

In [55]:
df_village_list[df_village_list['Village Name'] == 'kuther']

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
1487,2014-2017,himachal pradesh,jwali/kangra,ii,kuther,16,kuther


In [61]:
df_village_list[df_village_list['Village Name'] == 'kashipur']

,Year,State Name,District Name,Zone No,Tehsil Name,Tehsil Code,Village Name
893,2011-2014,orissa,rayagada,2,kashipur,21,kashipur


In [58]:
df_SOI_shape_filtered.sample(5)

,NAME,dtnmcoc,state,tehnmcoc,geometry,similarity
328649,balasar,nagaur,rajasthan,nagaur,"POLYGON ((73.62862 27.32284, 73.63861 27.32345...",1.000000
527441,jalalpur,hardoi,uttarpradesh,hardoi,"POLYGON ((80.24526 27.52287, 80.24443 27.5194,...",1.000000
240862,jainpur,jamtara,jharkhand,k vidyasagar,"POLYGON ((86.73543 24.03868, 86.73524 24.03825...",0.952381
336035,kashipur,sawai madhopur,rajasthan,gangapur,"POLYGON ((76.75386 26.56393, 76.76088 26.56423...",1.000000
446655,devhara,shahdol,madhya pradesh,anuppur,"POLYGON ((81.58911 23.16918, 81.59261 23.17042...",0.952381


In [ ]:
df_SOI_shape_filtered = df_SOI_shape_filtered.drop(columns=['similarity'])

In [64]:
def calculate_jaro_similarity(village_name1, village_name2):
    if pd.isnull(village_name1) or pd.isnull(village_name2):
        return 0 
    return jaro_similarity(village_name1, village_name2)

In [66]:
df_SOI_shape['similarity'] = df_SOI_shape.apply(
    lambda row: max(
        [
            calculate_jaro_similarity(row['NAME'], village)
            for village in df_village_list[
                (df_village_list['State Name'].notnull()) & 
                (df_village_list['State Name'].str.lower() == row['state'].lower())
            ]['Village Name']
        ]
    ) if (pd.notnull(row['state']) and not df_village_list[
                (df_village_list['State Name'].notnull()) & 
                (df_village_list['State Name'].str.lower() == row['state'].lower())
            ]['Village Name'].empty) else 0, 
    axis=1
)

In [69]:
df_SOI_shape_filtered.shape

(6993, 6)

In [67]:
df_SOI_shape_filtered = df_SOI_shape[df_SOI_shape['similarity'] > 0.95]

In [ ]:
df_SOI_shape_filtered = df_SOI_shape_filtered.drop(columns=['similarity'])